# Extraire la legende depuis la description
## tentative n°1

https://regex101.com/

In [1290]:
import pickle
import regex   # permet overlapping matching

In [1291]:
def print_legend( legend ):
    sortfun = lambda x: ( x['number'] , len(x['position']), x['label'][::-1] )
    legendsorted = sorted( legend, key=sortfun  )

    current_number = legendsorted[0]['number']
    for match in legendsorted:
        if current_number != match['number']:
            print('     --')
            current_number = match['number']

        n_merged = len(match['position'])
        n_merged_str = '  ' if n_merged==1 else '(%i)'%n_merged
        print( '{label:>42} {number:>4}  {n}  {context:>30}'.format( **match, n=n_merged_str) )

In [1292]:
# load pre-processed data
data = pickle.load( open( "../web/data/patent_infos.pickle", "rb" ) )
len(data)

379

In [1293]:
# select a patent

k = 38 #44 #41
patent = list(data.values())[k]
description = patent['description']

print( patent['patent_number'] )
print( 'https://www.google.com/patents/%s'%patent['patent_number'] )

print('nombre de mots :', len(description.split(' ')))

US9266223
https://www.google.com/patents/US9266223
nombre de mots : 13975


In [1294]:
# Travail sur le texte entier :

description = description.replace('\n', '')
description = regex.sub('\s+',' ', description) # remove multiple space
description = description.lower()

# patent number XX
patent_number_pattern = r'(\d{1, 4}[,-])?\d{3, 8}[,-]\d{3, 8}'
description = regex.sub(patent_number_pattern, '', description  )

# remove non-AASII caracters
description = regex.sub(r'[^\x00-\x7F]+', '', description)

In [1295]:
# Find values
units = ['inch', 'volt', 'm', 'mm']
find_sci_values =  r" (\d{1,4}([\.,]\d{1,4})? (%s)\W)" % '|'.join(units)

print( regex.findall(find_sci_values, description) )
description = regex.sub(find_sci_values, '<value>', description  )

[]


In [1296]:
# Find values
units = ['inch', 'volt', 'm', 'mm']
find_sci_values =  r" (\d{1,4}([\.,]\d{1,4})? (%s)\W)" % '|'.join(units)

print( regex.findall(find_sci_values, description) )
description = regex.sub(find_sci_values, '<value>', description  )

[]


In [1297]:
# Find floats
find_sci_floats =  r" (\d{1,4}([\.,]\d{1,4})\W)" 

print( regex.findall(find_sci_floats, description) )
description = regex.sub(find_sci_floats, '<value>', description  )

[('7,717,', ',717'), ('5,062,', ',062'), ('3,742,', ',742'), ('2,020,', ',020')]


In [1298]:
# " between 0.4 to 2.5 " 
betweenpattern = r"(between [\d\.,]{1,4} (to|and) [\d\.,]{1,4}\W)"

print( regex.findall(betweenpattern, description) )
description = regex.sub(betweenpattern, '<between>', description  )

[]


In [1299]:
description

'cross-reference to related applications this application is a continuation-in-part of u.s. design application ser. no. 29/467,935 filed concurrently on sep. 25, 2013, which application is a continuation-in-part of international application no. pct/us2009/001442, filed mar. 4, 2009 designating the united states and other countries, which is a continuation of u.s. application ser. no. 12/137,482, filed jun. 11, 2008, now u.s. pat. no.<value>017 issued on may 18, 2010, the disclosures of which are hereby incorporated by reference in their entirety to provide continuity of disclosure to the extent such disclosures are not inconsistent with the disclosure herein. field of the invention this invention generally relates to tools and more particularly relates to hand tools and methods of manufacturing and using same. background of the invention conventional hand tools, such as conventional fingernail and toe nail clippers, have proven problematic to use, particularly when used by the elderly,

In [1300]:
# trouve tout les chiffres, avec un espace devant
#exp =  r"(.{40}) (\d{1,3})"

exp =  r" (?P<n1>\d{1,3})( (and|or) (?P<n2>\d{1,3}))?[^a-z0-9\-]"

# exp =  r'(\d{1,3}((, | and )\d{1,3})*)'   # E(SE)  Element  Separator

allmatches = list( regex.finditer(exp, description) )
print(len(allmatches))

legend = []
for m in allmatches:

    label = description[ max(0, m.start()-40 ): m.start() ]  # to prevent overlapping
    context = description[ max(0, m.start()-10 ): min( len(description), m.end()+10 ) ]  # to debug
    
    item = {'number':int(m.group('n1')), 'label':label, 'position':[m.span('n2')], 'context':context}
    legend.append( item )
    
    if m.group('n2'): # correspond to "23 and 24"
        item = {'number':int(m.group('n2')), 'label':label, 'position':[m.span('n2')], 'context':context}
        legend.append( item )
        
print(len(legend))

142
158


In [1301]:
# autre approche
#    ' 45, 46 and 47 '

exp =  r' (\d{1,3}(( ?, ?| and | or )\d{1,3})*)[^a-z0-9\-]'

# explication:
# la forme est E(SE) avec Element  Separator
# et le séparateur est une virgule ou and ou or 

splitpat = regex.compile(r'[^\d]+')

allmatches = list( regex.finditer(exp, description) )

legend = []
for m in allmatches:
    capture = m.group(1)
    numbers = splitpat.split( capture )
    start = m.start()
    
    label = description[ max(0, m.start()-40 ): m.start() ]  # to prevent overlapping
    context = description[ max(0, m.start()-10 ): min( len(description), m.end()+10 ) ]  # to debug
    
    for n in numbers:
        item = {'number':int(n), 'label':label, 'position':[m.span(1)], 'context':context}
        legend.append( item )
        
print(len(legend))

158


In [1302]:
def disqualify( legend, pattern, n=999 ):
    for i, row in enumerate( legend ):
        if pattern.search( row['label'] ):
            legend[ i ]['number'] = n

# Disqualifie si point final :  & tiret   ... mais pas une virgule car 45, 46 and 47 -> on va le faire avant ...
# Fig. 1  ;  No. 511  
# en fait si ce n'est pas un Word caracter... que l'on restreint à l'alphabet, pour éviter les symboles
pattern = regex.compile( r'[^a-z]$', regex.I )

disqualify( legend, pattern, n=999 )

        
# si c'est un mois, ou un autre mots clés :
keywords = ['january', 'february', 'march', 'april', 'may', 'june',\
          'july', 'august', 'september', 'october', 'november', 'december',\
           'figure', 'figures', 'claim', 'claims', 'at', 'and', 'numeral']

keywords.extend( ['the', 'a', 'an', 'these', 'their', 'when', 'with', 'by', 'this', \
               'have', 'having', 'has', 'is', 'are', 'over', 'its', 'of said', 'and', 'as',\
              'of', 'in', 'to', 'but', 'another', 'through', 'on'] )

pattern = regex.compile( r'\W(%s)$' % '|'.join(keywords), regex.I )

disqualify( legend, pattern, n=998 )
        
    
print_legend( legend )

  st of one of the parts turns (see column    2             see column 2, lines 15-
     --
  e and tilt a limited amount (i.e., about   30            .e., about 30) in the x 
  e and tilt a limited amount (e.g., about   30            .g., about 30) in the x 
     --
  ate the reference characters by at least  100           y at least 100 unitsthis 
     --
  e 1595 1596 a smooth lower connector pin  159           nector pin 159 1610 a low
     --
  o swivel or rotate side-to-side at least  180           e at least 180 degrees in
  simultaneously swivel or rotate at least  180           e at least 180 degrees in
  han or equal to an angle theta  of about  180             of about 180 degrees. t
   cutting edges through an angle of about  180           e of about 180. the upper
     --
   a contoured, arcuate-shaped lower shell  482           ower shell 482 1484 an ar
     --
   ). referring again to figs. 1 , 2 , and  998              , 2 , and 3 , upper ha
  erring to figs. 1 , 4 , 4 

## Coupe sur certain mot clé (the, a, an, ...)

In [1303]:
def cut_label( legend, pattern ):
    for i, row in enumerate( legend ):
        label = row['label']

        match = pattern.search(label)

        if match:
            newlabel = match.group(2)
            legend[i]['label'] = newlabel

In [1304]:
small_words = ['the', 'a', 'an', 'these', 'their', 'when', 'with', 'by', 'this', 'that', \
               'have', 'having', 'has', 'is', 'are', 'should', 'over', 'its', 'of said', 'and', 'as',\
              'of', 'in', 'to', 'at',  'but', 'another', 'through', 'on', 'same', 'from',\
              'include', 'includes', 'beyond', 'between']

pattern = regex.compile( r'^.* (%s) (.+)$' % '|'.join(small_words), regex.I )

cut_label( legend, pattern )

In [1305]:
# cut at point and comma :
pattern = regex.compile( '^.*([\.,;] )(.+)$' )

cut_label( legend, pattern )

print_legend( legend )

                   parts turns (see column    2             see column 2, lines 15-
     --
                                     about   30            .g., about 30) in the x 
                                     about   30            .e., about 30) in the x 
     --
                                     least  100           y at least 100 unitsthis 
     --
                smooth lower connector pin  159           nector pin 159 1610 a low
     --
                                     least  180           e at least 180 degrees in
                                     least  180           e at least 180 degrees in
                                     about  180           e of about 180. the upper
                                     about  180             of about 180 degrees. t
     --
                arcuate-shaped lower shell  482           ower shell 482 1484 an ar
     --
                                       and  998              , 2 , and 3 , upper ha
                            

In [1306]:
def get_positions( label, candidats ):
    positions = []
    for item in candidats:
        if item['label']==label:
            positions.extend( item['position'] )
    return positions

def get_context( label, candidats ):
    items = [ item for item in candidats if item['label']==label ]
    
    return items[0]['context']

def merge( legend ):
    # merge identical label
    
    numbers_unique = { item['number'] for item in legend  } # set

    new_legend = []
    for n in numbers_unique:

        candidats = [ line for line in legend if line['number']==n ]

        labels_unique = { item['label'] for item in candidats  }

        new_candidats = [ { 'number':n, 'label':label_u,\
                           'position':get_positions(label_u, candidats), 'context':get_context(label_u, candidats) }
                          for label_u in labels_unique ]
        
        new_legend.extend( new_candidats )

    return new_legend

In [1307]:
print_legend(  merge( legend ) )

                   parts turns (see column    2             see column 2, lines 15-
     --
                                     about   30  (2)        .g., about 30) in the x 
     --
                                     least  100           y at least 100 unitsthis 
     --
                smooth lower connector pin  159           nector pin 159 1610 a low
     --
                                     least  180  (2)       e at least 180 degrees in
                                     about  180  (2)       e of about 180. the upper
     --
                arcuate-shaped lower shell  482           ower shell 482 1484 an ar
     --
                                       may  998            ued on may 18, 2010, the
                                       the  998  (2)       efore, the 180 degree rot
                                       and  998  (7)          , 2 , and 3 , upper ha
     --
                             wherein: fig.  999             rein: fig. 1 is a view 
   mounting or 

### To look
* line 5 - 5
*      march   15                 march 15,   
en debut de la description ...
* sort of recursive... identifier un permet d'en déduire un autre...
* " respectivement A et B " ... respective
* letter sur les vieux brevets
* merge keywords and small_words ??
* pluriel

## other things
* Find values like "110 volts"... 